### Setup a Conda Environment on Coiled

In [ ]:
import coiled

coiled.create_software_environment(
   name='mapshader-tiling',
   conda={
       'channels': ['conda-forge', 'defaults'],
       'dependencies': [
           'python=3.9',
           'mapshader',
           'dask=2022.04.2',
           'distributed=2022.4.2',
           'cloudpickle=2.0.0',
           'spatialpandas',
           'boto3',
       ],
    },
)

### Create Dask Cluster on Coiled 

In [ ]:
from coiled.v2 import Cluster
cluster = Cluster(name='mapshader-tiler',
                  n_workers=10,
                  worker_cpu=2,
                  worker_options={"nthreads": 1},
                  scheduler_memory="8 GiB",
                  software='mapshader-tiling')

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

### Clear cluster memory if necessary

In [ ]:
client.restart()

## Tile World Cities (Sparse Points)

### Setup Mapshader Source

In [1]:
from os import path

import geopandas as gpd
import mapshader
import spatialpandas

from mapshader.sources import RasterSource


def world_elevation_source():

    # find data path
    elevation_path = './elevation.tif'

    # construct transforms
    squeeze_transform = dict(name='squeeze', args=dict(dim='band'))
    cast_transform = dict(name='cast', args=dict(dtype='float64'))
    orient_transform = dict(name='orient_array')
    flip_transform = dict(name='flip_coords', args=dict(dim='y'))
    reproject_transform = dict(name='reproject_raster', args=dict(epsg=3857))

    transforms = [squeeze_transform,
                  cast_transform,
                  orient_transform,
                  flip_transform,
                  reproject_transform]
    
    # construct value obj
    source_obj = dict()
    source_obj['name'] = 'Elevation'
    source_obj['key'] = 'elevation'
    source_obj['text'] = 'Elevation'
    source_obj['description'] = 'Global Elevation Dataset'
    source_obj['geometry_type'] = 'raster'
    source_obj['shade_how'] = 'linear'
    source_obj['cmap'] = ['white', 'black']
    source_obj['span'] = 'min/max'
    source_obj['raster_padding'] = 0
    source_obj['raster_interpolate'] = 'linear'
    source_obj['xfield'] = 'geometry'
    source_obj['yfield'] = 'geometry'
    source_obj['filepath'] = elevation_path
    source_obj['transforms'] = transforms
    source_obj['service_types'] = ['tile', 'wms', 'image', 'geojson']

    source_obj['tiling'] = dict(min_zoom=1,
                                max_zoom=2,
                                xmin_field='buffer_0_4326_xmin',
                                xmax_field='buffer_0_4326_xmax',
                                ymin_field='buffer_0_4326_ymin',
                                ymax_field='buffer_0_4326_ymax'
                               )

    return source_obj


elevation_source = RasterSource.from_obj(world_elevation_source())
elevation_source.load()
elevation_source.data

Not Absolute
# ----------------------
# APPLYING TRANSFORMS Elevation
# ----------------------
	Applying squeeze
	Applying cast
	Applying orient_array
	Applying flip_coords
	Applying reproject_raster


Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.


<xarray.DataArray '/Users/thuydo/makepath/work/mapshader/examples/elevation.tif' (
                                                                                  y: 515,
                                                                                  x: 315)>
array([[147., 147., 147., ..., 145., 145., 147.],
       [147., 147., 147., ..., 145., 145., 147.],
       [147., 147., 147., ..., 145., 145., 147.],
       ...,
       [204., 202., 202., ..., 200., 200., 202.],
       [204., 202., 202., ..., 200., 200., 202.],
       [204., 202., 202., ..., 200., 200., 202.]])
Coordinates:
  * x            (x) float64 -1.997e+07 -1.985e+07 ... 1.978e+07 1.99e+07
  * y            (y) float64 3.276e+07 3.264e+07 ... -3.259e+07 -3.272e+07
    spatial_ref  int64 0
Attributes:
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    _FillValue:     nan

### Generate tile images and save to an S3 bucket

In [ ]:
from mapshader.tile_utils import save_tiles_to_outpath

save_tiles_to_outpath(elevation_source, outpath='s3://mapshader-tiling-test-999/elevation/')

all_tiles [{'x': 0, 'y': 1, 'z': 1, 'q': '2'}, {'x': 1, 'y': 1, 'z': 1, 'q': '3'}, {'x': 0, 'y': 0, 'z': 1, 'q': '0'}, {'x': 1, 'y': 0, 'z': 1, 'q': '1'}, {'x': 0, 'y': 3, 'z': 2, 'q': '22'}, {'x': 1, 'y': 3, 'z': 2, 'q': '23'}, {'x': 2, 'y': 3, 'z': 2, 'q': '32'}, {'x': 3, 'y': 3, 'z': 2, 'q': '33'}, {'x': 0, 'y': 2, 'z': 2, 'q': '20'}, {'x': 1, 'y': 2, 'z': 2, 'q': '21'}, {'x': 2, 'y': 2, 'z': 2, 'q': '30'}, {'x': 3, 'y': 2, 'z': 2, 'q': '31'}, {'x': 0, 'y': 1, 'z': 2, 'q': '02'}, {'x': 1, 'y': 1, 'z': 2, 'q': '03'}, {'x': 2, 'y': 1, 'z': 2, 'q': '12'}, {'x': 3, 'y': 1, 'z': 2, 'q': '13'}, {'x': 0, 'y': 0, 'z': 2, 'q': '00'}, {'x': 1, 'y': 0, 'z': 2, 'q': '01'}, {'x': 2, 'y': 0, 'z': 2, 'q': '10'}, {'x': 3, 'y': 0, 'z': 2, 'q': '11'}]
tiles_df     x  y  z   q
2   0  0  1   0
0   0  1  1   2
3   1  0  1   1
1   1  1  1   3
16  0  0  2  00
12  0  1  2  02
8   0  2  2  20
4   0  3  2  22
17  1  0  2  01
13  1  1  2  03
9   1  2  2  21
5   1  3  2  23
18  2  0  2  10
14  2  1  2  12
10  

### View tiles on OSM basemap

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles

tiles_url = 'https://mapshader-tiling-test-999.s3.amazonaws.com/elevation/{z}/{x}/{y}.png'
tile_layer=TileLayer(url=tiles_url)

from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    zoom=4,
    scroll_wheel_zoom=True)

m.add_layer(tile_layer)
m
display(m)